In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from dotenv import load_dotenv
import requests
import openai 
import shutil
import os

os.environ['OPENAI_API_KEY'] = 'key'

In [2]:
# Fetch data from Spotify API
url = "https://spotify23.p.rapidapi.com/search/"

querystring = {"q":"metallica", "type": "album", "limit": "50"}

headers = {
	"x-rapidapi-key": "31889f1ddfmsh96d42e48a3f8968p1484b5jsn5f8010587ad9",
	"x-rapidapi-host": "spotify23.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Extract albums from the JSON response
results = response.json().get('albums', {}).get('items', [])
print(results)

[{'data': {'uri': 'spotify:album:1nTvIQEXvygqSIqc2vuwAz', 'name': 'Ride The Lightning (Remastered)', 'artists': {'items': [{'uri': 'spotify:artist:2ye2Wgw4gimLv2eAKyk1NB', 'profile': {'name': 'Metallica'}}]}, 'coverArt': {'sources': [{'url': 'https://i.scdn.co/image/ab67616d00001e02b09e5084136821bf64d327f4', 'width': 300, 'height': 300}, {'url': 'https://i.scdn.co/image/ab67616d00004851b09e5084136821bf64d327f4', 'width': 64, 'height': 64}, {'url': 'https://i.scdn.co/image/ab67616d0000b273b09e5084136821bf64d327f4', 'width': 640, 'height': 640}]}, 'date': {'year': 1984}}}, {'data': {'uri': 'spotify:album:55fq75UfkYbGMq4CncCtOH', 'name': 'Metallica (Remastered)', 'artists': {'items': [{'uri': 'spotify:artist:2ye2Wgw4gimLv2eAKyk1NB', 'profile': {'name': 'Metallica'}}]}, 'coverArt': {'sources': [{'url': 'https://i.scdn.co/image/ab67616d00001e021f9edf15e43f4c2f4938b869', 'width': 300, 'height': 300}, {'url': 'https://i.scdn.co/image/ab67616d000048511f9edf15e43f4c2f4938b869', 'width': 64, 'he

In [3]:
# Create documents
all_albums = []
titles = []
for result in results:
    release_year = result['data'].get('date', {}).get('year', '')
    album = {}
    if release_year:
        album['artist'] = 'Metallica'
        album['album'] = result['data'].get('name', '')
        album['year'] = release_year
         
        title = album['album']
        if title not in titles:
            titles.append(title)
            all_albums.append(album)

In [4]:
# Function to filter albums by year
def filter_albums_by_year(all_albums, year_filter):
    filtered_documents = [album for album in all_albums if int(album['year']) > year_filter]
    
    if not filtered_documents:
        print("No albums found after the specified year.")
    else:
        filtered_documents_total = len(filtered_documents)
        print(f'It was found {filtered_documents_total} albums:')
        for album in filtered_documents:
            print(f" - {album['year']}: {album['album']}.")
            

In [5]:
year_filter = 2007        
filter_albums_by_year(all_albums, year_filter)

It was found 17 albums:
 - 2024: Plays Metallica, Vol. 2.
 - 2023: 72 Seasons.
 - 2013: Metallica Through The Never (Music from the Motion Picture).
 - 2008: Death Magnetic.
 - 2021: The Metallica Blacklist.
 - 2016: Hardwired…To Self-Destruct.
 - 2023: metallica.
 - 2020: Metallica Remixes.
 - 2023: Metallica.
 - 2016: Hardwired…To Self-Destruct (Deluxe).
 - 2012: Metallica Para Bebes.
 - 2016: Plays Metallica by Four Cellos (2016 Remastered Version).
 - 2023: Metallica Lullabies.
 - 2023: Metallica meets Billie Eilish.
 - 2011: Lullaby Versions of Metallica.
 - 2022: Metallica (Remix).
 - 2020: S&M2.
